In [1]:
# Загрузка библиотек
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from funcs import fill_na_with_similar, fill_with_mode

In [2]:
# Загрузка датасета
db3 = pd.read_excel('Database_3.xlsx')
db3.head()

,No,Year,Material,Type,Shape,Coat/Functional Group,Synthesis_Method,Surface_Charge,Diameter (nm),Size_in_Water (nm),...,Cell Line_Primary Cell,Time (hr),Concentration (ug/ml),Test,Test_Indicator,Aspect_Ratio,Cell_Viability (%),PDI,Article_ID,DOI
0,1,2015,SiO2,I,Sphere,CTAB,Precipitation,Negative,NaN,386.9,...,L,12,1.95,MTT,TetrazoliumSalt,NaN,113.670,NaN,1,10.3109/15376516.2015.1070229
1,2,2015,SiO2,I,Sphere,CTAB,Precipitation,Negative,NaN,386.9,...,L,12,3.90,MTT,TetrazoliumSalt,NaN,97.265,NaN,1,10.3109/15376516.2015.1070229
2,3,2015,SiO2,I,Sphere,CTAB,Precipitation,Negative,NaN,386.9,...,L,12,7.80,MTT,TetrazoliumSalt,NaN,10873.000,NaN,1,10.3109/15376516.2015.1070229
3,4,2015,SiO2,I,Sphere,CTAB,Precipitation,Negative,NaN,386.9,...,L,12,15.60,MTT,TetrazoliumSalt,NaN,98.451,NaN,1,10.3109/15376516.2015.1070229
4,5,2015,SiO2,I,Sphere,CTAB,Precipitation,Negative,NaN,386.9,...,L,12,31.20,MTT,TetrazoliumSalt,NaN,98.056,NaN,1,10.3109/15376516.2015.1070229


In [3]:
# Удаление ненужных столбцов
db3.drop(['Year', 'No', 'Article_ID', 'DOI'], axis=1, inplace=True)

In [4]:
# Очищение столбца с материалом НЧ
db3['Material'].unique()

array(['SiO2', 'QD', 'IronOxide', 'Bi', 'Au', 'Ag', 'TiO2', 'Dendrimer',
       'Dendrmer', 'PLGA', 'Alginate', 'PTFE-PMMA', 'ZnO', 'Pt', 'CuO',
       'CeO2', 'Co3O4', 'MWCNT', 'HAP', 'MgO', 'Polystyrene', 'Ni',
       'Chitosan', 'Al2O3', 'SLN', 'EudragitRL', 'Carbon', 'ZrO2', 'C60',
       'C70', 'SWCNT', 'Diamond', 'Graphite', 'Fe2O3', 'IronOide', 'NiO',
       'Ay', 'Cu', 'Ce O2', 'Bi2O3'], dtype=object)

In [5]:
db3['Material'].replace({
    'IronOxide': 'Fe2O3',
    'IronOide': 'Fe2O3',
    'Ay': 'Au'
}, inplace=True)

In [7]:
# Очищение столбца органичности материала
db3['Type'].unique()

array(['I', 'O', 'C', 0], dtype=object)

In [8]:
db3['is_inorganic'] = (db3['Type'] == 'I').astype('int32')
db3.drop('Type', axis=1, inplace=True)

In [9]:
db3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4111 entries, 0 to 4110
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Material                  4111 non-null   object 
 1   Shape                     4111 non-null   object 
 2   Coat/Functional Group     4110 non-null   object 
 3   Synthesis_Method          4111 non-null   object 
 4   Surface_Charge            3112 non-null   object 
 5   Diameter (nm)             3278 non-null   float64
 6   Size_in_Water (nm)        2330 non-null   float64
 7   Size_in_Medium (nm)       1989 non-null   float64
 8   Zeta_in_Water (mV)        2487 non-null   object 
 9   Zeta_in_Medium (mV)       1670 non-null   float64
 10  Cell_Type                 4111 non-null   object 
 11  No_of_Cells (cells/well)  3684 non-null   float64
 12  Human_Animal              4111 non-null   object 
 13  Cell_Source               4110 non-null   object 
 14  Cell_Tis

In [10]:
# Выявление ошибочных значений
for i in db3['Zeta_in_Water (mV)']:
    if type(i) not in [int, float]:
        print(type(i), i)

<class 'str'> 15O


In [11]:
db3['Zeta_in_Water (mV)'].replace({
    '15O': 150
}, inplace=True)

In [12]:
# Попытка понять, от чего зависит колонка "Surface_Charge"
for col in ['Zeta_in_Water (mV)', 'Zeta_in_Medium (mV)']:
    correct = 0
    incorrect = 0

    ids = np.where(1 - (db3[col].isna() | db3['Surface_Charge'].isna()))[0]

    for idx in ids:
        if db3['Surface_Charge'][idx] == 'Positive' and db3[col][idx] >= 0.25:
            correct += 1
        elif db3['Surface_Charge'][idx] == 'Negative' and db3[col][idx] <= -0.25:
            correct += 1
        elif db3['Surface_Charge'][idx] == 'Neutral' and abs(db3[col][idx]) < 0.25:
            correct += 1
        else:
            incorrect += 1
    
    print(f'Score for "{col}" is {correct / (correct + incorrect)}')

Score for "Zeta_in_Water (mV)" is 0.9798873692679002
Score for "Zeta_in_Medium (mV)" is 0.9479041916167664


In [13]:
# Сильно зависит от обоих колонок, можно удалить
db3.drop('Surface_Charge', axis=1, inplace=True)

In [14]:
# Корреляция двух похожих колонок
db3.filter(['Zeta_in_Water (mV)', 'Zeta_in_Medium (mV)']).dropna().corr()

,Zeta_in_Water (mV),Zeta_in_Medium (mV)
Zeta_in_Water (mV),1.000000,0.459561
Zeta_in_Medium (mV),0.459561,1.000000


In [15]:
# Заполнение пропусков
db3['Coat/Functional Group'].fillna('None', inplace=True)

In [16]:
# Удаление ненужных колонок
db3.drop(['No_of_Cells (cells/well)', 'Cell_Source', 'PDI'], axis=1, inplace=True)

In [17]:
# Заполнение пустот
for col in db3.columns[db3.isna().any()]:
    score = fill_na_with_similar(db3, col, key='Material')
    fill_with_mode(db3[col])
    print(f'Score for "{col}" is {round(score * 100, 2)}%')

Score for "Diameter (nm)" is 79.71%
Score for "Size_in_Water (nm)" is 86.75%
Score for "Size_in_Medium (nm)" is 81.39%
Score for "Zeta_in_Water (mV)" is 89.04%
Score for "Zeta_in_Medium (mV)" is 79.31%
Score for "Concentration (ug/ml)" is 53.15%
Score for "Aspect_Ratio" is 48.41%


In [18]:
# Замена некоторых названий материалов
db3['Material'].unique()

array(['SiO2', 'QD', 'Fe2O3', 'Bi', 'Au', 'Ag', 'TiO2', 'Dendrimer',
       'Dendrmer', 'PLGA', 'Alginate', 'PTFE-PMMA', 'ZnO', 'Pt', 'CuO',
       'CeO2', 'Co3O4', 'MWCNT', 'HAP', 'MgO', 'Polystyrene', 'Ni',
       'Chitosan', 'Al2O3', 'SLN', 'EudragitRL', 'Carbon', 'ZrO2', 'C60',
       'C70', 'SWCNT', 'Diamond', 'Graphite', 'NiO', 'Cu', 'Ce O2',
       'Bi2O3'], dtype=object)

In [19]:
db3['Material'].replace({
    'Dendrmer': 'Dendrimer',
    'MWCNT': 'Carbon NT',
    'SWCNT': 'Carbon NT',
    'Ce O2': 'CeO2'
}, inplace=True)

In [20]:
# Сохранение таблицы
db3.to_excel('proc_db3.xlsx', index=False)